In [1]:
from dataclasses import dataclass

# Dataclasses for SDK5 Configuration inputs parameteres.
@dataclass
class ChirpComnCfg:
    digOutputSampRate: float # Sampling Rate decimator (sampling rate given by 100MHz/digoutputsamprate i.e digoutsamplerate = 8, sampling rate = 12.5MHz)
    digOutputBitsSel: int # Digital output sample bits select:
    # n important pelo momento
    """ 
        - 0 - Digital sample output is 12 MSB bits of DFE after rounding 4 LSBs. 
        - 1 - Digital sample output is 12 bits after rounding 3 LSBs & clipping 1 MSB. 
        - 2 - Digital sample output is 12 bits after rounding 2 LSBs & clipping 2 MSB. 
        - 3 - Digital sample output is 12 bits after rounding 1 LSBs & clipping 3 MSB. 
        - 4 - Digital sample output is 12 LSB bits after clipping 4 MSB. 
        - 5 - Digital sample output is 16 bits.
    """
    dfeFirSel: int # The final stage FIR filter's characteristics:
    # n important pelo momento
    """
        0 - Long Filter (90% visibility)
        1 - Shorter filter (80% visibility)
    """
    numOfAdcSamples: int # Number of ADC samples collected during ADC sampling time (needs to be power of 2)
    
    chirpTxMimoPatSel: int # ? modelos de estimacao de angulo diferentes (n importante no momento)
    """
        1 - enable TDM
        4 - enable BPM
    """
    
    chirpRampEndTime: float # ramp end time (us) 
    # bem importante !!!!!!!!!!!!!!!!!!!!!!!!
    """
        Ramp end time (us): This parameter is the sum of the ADC start time, ADC sampling time, and some ramp excess time.
        In other words, its the ramp period.
        Documentation says a minimum of:
        (chirpAdcSkipSamples + numOfAdcSamples + 7), for short filter (dfeFirSel = 1)
        (chirpAdcSkipSamples + numOfAdcSamples + 10), for long filter (dfeFirSel = 0)

    """
    
    chirpRxHpfSel: int # Chirp Profile HPF corner frequency:
    # High pass filter to filter out the bumper or anything too close to the radar.
    """
        - 0 - 175kHz HPF corner frequency 
        - 1 - 350kHz HPF corner frequency
        - 2 - 700kHz HPF corner frequency 
        - 3 - 1400kHz HPF corner frequency
    """

@dataclass
class ChirpTimingCfg:
    chirpIdleTime: float # Idle Time (us) in the chirp cicle before the frequency ramp
    chirpAdcSkipSamples: int # Number of samples skipped during ADC sampling. The ADC start time in us is computed based on the sampling rate. (0 - 63)
    chirpTxStartTime: float # Tx Start Time (us). Recommended Range: +- 5us
    chirpRfFreqSlope: float # Frequency Slope (MHz/us). (-399MHz/us to +399MHz/us)
    chirpRfFreqStart: float # Starting Frequency (GHz)

@dataclass
class ChannelCfg:
    rxChannelEn: float # Rx Antenna Mask; for 3 antennas, it is 0111b = 7
    txChannelEn: float # Tx Antenna; for 2 antenas it is 0011b = 3
    miscCtrl: float # ? SoC cascading (not supported with SDK)

@dataclass
class FrameCfg:
    numOfChirppsInBurst: int # Number of chirps in a burst (1 - 65535)
    numOfChirpsAccum: int # Number of chirps accumulated (0 - 64)
    burstPeriodicity: float # Burst periodicity (us)
    numOfBurstsInFrame: int # Number of bursts within a frame (1 - 4096)
    framePeriodicity: float # Frame Periodicity (ms)
    numOfFrames: int # Number of total frames (0 - 65535) (0 = infinite)

@dataclass
class FactoryCalibCfg:
    saveEnable: bool # Save calibration data to flash (0-1)
    restoreEnable: bool # Restore and load calibration data from FLASH
    rxGain: float # RX Gain in dB (30-40 strongly recommended)
    txBackoff: float # TX Backoff in dB (0 - 26)
    flashOffset: float # Address ofset in the flash to be used while saving or restoring calibration data


In [2]:
# Function to parse the file and map values to the respective dataclasses
def parse_cfg_file(file_path):
    chirpComnCfg = None
    chirpTimingCfg = None
    channelCfg = None
    frameCfg = None
    factoryCalibCfg = None
    
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    for line in lines:
        # Skip comments or empty lines
        if line.startswith('%') or not line.strip():
            continue

        parts = line.strip().split()
        config_type = parts[0]
        values = list(map(float, parts[1:]))
        
        if config_type == 'chirpComnCfg':
            chirpComnCfg = ChirpComnCfg(*values)
        elif config_type == 'chirpTimingCfg':
            chirpTimingCfg = ChirpTimingCfg(*values)
        elif config_type == 'channelCfg':
            channelCfg = ChannelCfg(*values)
        elif config_type == 'frameCfg':
            frameCfg = FrameCfg(*values)
        elif config_type == 'factoryCalibCfg':
            factoryCalibCfg = FactoryCalibCfg(*values)
    
    return chirpComnCfg, chirpTimingCfg, channelCfg, frameCfg, factoryCalibCfg

In [3]:
file_path = './profile_2024_10_10T13_21_31_415.cfg'
chirpComnCfg, chirpTimingCfg, channelCfg, frameCfg, factoryCalibCfg = parse_cfg_file(file_path)

print("ChirpComnCfg:", chirpComnCfg)
print("ChirpTimingCfg:", chirpTimingCfg)
print("ChannelCfg:", channelCfg)
print("FrameCfg:", frameCfg)
print("FactoryCalibCfg:", factoryCalibCfg)

ChirpComnCfg: ChirpComnCfg(digOutputSampRate=33.0, digOutputBitsSel=0.0, dfeFirSel=0.0, numOfAdcSamples=128.0, chirpTxMimoPatSel=4.0, chirpRampEndTime=50.0, chirpRxHpfSel=0.0)
ChirpTimingCfg: ChirpTimingCfg(chirpIdleTime=6.0, chirpAdcSkipSamples=19.0, chirpTxStartTime=0.0, chirpRfFreqSlope=18.39, chirpRfFreqStart=76.0)
ChannelCfg: ChannelCfg(rxChannelEn=7.0, txChannelEn=3.0, miscCtrl=0.0)
FrameCfg: FrameCfg(numOfChirppsInBurst=2.0, numOfChirpsAccum=0.0, burstPeriodicity=227.0, numOfBurstsInFrame=20.0, framePeriodicity=200.0, numOfFrames=0.0)
FactoryCalibCfg: FactoryCalibCfg(saveEnable=1.0, restoreEnable=0.0, rxGain=30.0, txBackoff=0.0, flashOffset=0.0)


In [4]:
# Important parameters that come directly from the radar configuration (left here as comment as a path remember)
    # slope = chirpTimingCfg.chirpRfFreqSlope # MHz/us
    # f0 = chirpTimingCfg.chirpRfFreqStart # GHz
    # nADC = chirpComnCfg.numOfAdcSamples # number of samples taken with the ADC (always power of two)
    # T_chirp_ramp = chirpComnCfg.chirpRampEndTime # us # ramp duration
    # chirp_idle_time = chirpTimingCfg.chirpIdleTime # us
    
    # n_skiped_chirps = chirpTimingCfg.chirpAdcSkipSamples #skiped samples
    # n_chirps_in_burst = frameCfg.numOfChirppsInBurst
    # n_bursts_in_frame = frameCfg.numOfBurstsInFrame
    # T_frame = frameCfg.framePeriodicity # ms
    # T_burst = frameCfg.burstPeriodicity # us
    
# Important parameters that dont come directly from the radar configuration
SamplingFrequency = 100 / chirpComnCfg.digOutputSampRate # MHz # ADC freq
IntermediateFrequencyMaximum = 0.9 * SamplingFrequency / 2 # MHz # Maximum readable frequency from the ADC in the current config
TSampling = chirpComnCfg.numOfAdcSamples / SamplingFrequency # us # Period of sampling
EndFreq = chirpTimingCfg.chirpRfFreqStart + (chirpTimingCfg.chirpRfFreqSlope * chirpComnCfg.chirpRampEndTime * 1e-3) # GHz
Bandwidth = chirpTimingCfg.chirpRfFreqSlope * TSampling # MHz
TChirp = chirpComnCfg.chirpRampEndTime + chirpTimingCfg.chirpIdleTime # us
nChirpsLoop = frameCfg.numOfChirppsInBurst * frameCfg.burstPeriodicity # n chirps per frame
ADCValidStartTime = chirpTimingCfg.chirpAdcSkipSamples / SamplingFrequency # us
CarrierFreq = chirpTimingCfg.chirpRfFreqStart + ((chirpTimingCfg.chirpRfFreqSlope * ADCValidStartTime) + (Bandwidth / 2)) * 1e-3 # GHz
nRxAntenas = bin(int(channelCfg.rxChannelEn)).count("1")
nTxAntenas = bin(int(channelCfg.txChannelEn)).count("1")
nVirtualAntenas = nRxAntenas * nTxAntenas

In [5]:
import ipywidgets as widgets
from ipywidgets import interact

# Define the range of powers of two
min_power = 2  # 2^0
max_power = 11  # 2^10
# Generate the list of powers of two
powers_of_two = [2**i for i in range(min_power, max_power + 1)]

def update_parameters(
    chirpRfFreqSlope, chirpRfFreqStart, numOfAdcSamples, chirpRampEndTime, chirpIdleTime,
    chirpAdcSkipSamples, numOfChirppsInBurst, numOfBurstsInFrame, framePeriodicity, burstPeriodicity,
    digOutputSampRate
):
    c = 3e2 # m/us
    
    # Important parameters derived from radar configuration
    SamplingFrequency = 100 / digOutputSampRate # MHz
    IntermediateFrequencyMaximum = 0.9 * SamplingFrequency / 2 # MHz
    TSampling = numOfAdcSamples / SamplingFrequency # us
    EndFreq = chirpRfFreqStart + (chirpRfFreqSlope * chirpRampEndTime * 1e-3) # GHz
    Bandwidth = chirpRfFreqSlope * TSampling # MHz
    TChirp = chirpRampEndTime + chirpIdleTime # us
    nChirpsLoop = numOfChirppsInBurst * burstPeriodicity # n chirps per frame
    ADCValidStartTime = chirpAdcSkipSamples / SamplingFrequency # us
    CarrierFreq = chirpRfFreqStart + ((chirpRfFreqSlope * ADCValidStartTime) + (Bandwidth / 2)) * 1e-3 # GHz
    nTxAntenas = 1  # Assume 1 TX antenna for simplicity (adapt this based on your actual configuration)
    
    # Additional parameters calculated from the first code
    RangeMaximum = IntermediateFrequencyMaximum * c / (2 * chirpRfFreqSlope) # m
    RangeResolution = c / (2 * Bandwidth) # m
    MaximumUnambiguousVelocity = c / (4 * CarrierFreq * TChirp * 1e-3) # m/s
    VelocityResolution = 2 * MaximumUnambiguousVelocity / numOfBurstsInFrame # m/s
    RFDutyCycle = nChirpsLoop * TChirp / framePeriodicity * 1e-2
    ChirpRepetitionPeriod = burstPeriodicity if nTxAntenas == 4 else nTxAntenas * TChirp
    
    # Display the computed values
    print(f"{'Sampling Frequency:':<35} {SamplingFrequency:.2f} MHz")
    print(f"{'Intermediate Frequency Maximum:':<35} {IntermediateFrequencyMaximum:.2f} MHz")
    print(f"{'TSampling:':<35} {TSampling:.2f} us")
    print(f"{'End Frequency:':<35} {EndFreq:.2f} GHz")
    print(f"{'Bandwidth:':<35} {Bandwidth:.2f} MHz")
    print(f"{'TChirp:':<35} {TChirp:.2f} us")
    print(f"{'Chirps per Frame (nChirpsLoop):':<35} {nChirpsLoop:.2f}")
    print(f"{'ADC Valid Start Time:':<35} {ADCValidStartTime:.2f} us")
    print(f"{'Carrier Frequency:':<35} {CarrierFreq:.2f} GHz")
    print()
    print(f"{'Range Maximum:':<35} {RangeMaximum:.2f} m")
    print(f"{'Range Resolution:':<35} {RangeResolution:.2f} m")
    print(f"{'Maximum Unambiguous Velocity:':<35} {MaximumUnambiguousVelocity:.2f} m/s")
    print(f"{'Velocity Resolution:':<35} {VelocityResolution:.2f} m/s")
    print(f"{'Chirp Repetition Period:':<35} {ChirpRepetitionPeriod:.2f} us")
    print(f"{'RF Duty Cycle:':<35} {RFDutyCycle:.2f} %")
    print()

# Set a common layout for all sliders with a larger size
slider_layout = widgets.Layout(width='500px', height='40px')

chirpRampEndTime = widgets.FloatSlider(value=50, min=10, max=200, step=0.1, 
                                       description='chirpRampEndTime (us)', 
                                       style={'description_width': 'initial'}, layout=slider_layout)
chirpIdleTime = widgets.FloatSlider(value=6, min=0, max=100, step=0.1, 
                                    description='chirpIdleTime (us)', 
                                    style={'description_width': 'initial'}, layout=slider_layout)
chirpAdcSkipSamples = widgets.IntSlider(value=19, min=0, max=63, step=1, 
                                        description='chirpAdcSkipSamples', 
                                        style={'description_width': 'initial'}, layout=slider_layout)
numOfChirppsInBurst = widgets.IntSlider(value=2, min=1, max=65535, 
                                        description='numOfChirppsInBurst', 
                                        style={'description_width': 'initial'}, layout=slider_layout)
numOfBurstsInFrame = widgets.IntSlider(value=20, min=1, max=4096, 
                                       description='numOfBurstsInFrame', 
                                       style={'description_width': 'initial'}, layout=slider_layout)
framePeriodicity = widgets.FloatSlider(value=200, min=1, max=1000, step=1, 
                                       description='framePeriodicity (ms)', 
                                       style={'description_width': 'initial'}, layout=slider_layout)
burstPeriodicity = widgets.FloatSlider(value=227, min=1, max=1000, step=1, 
                                       description='burstPeriodicity (us)', 
                                       style={'description_width': 'initial'}, layout=slider_layout)
digOutputSampRate = widgets.FloatSlider(value=33, min=1, max=100, step=0.1, 
                                        description='digOutputSampRate', 
                                        style={'description_width': 'initial'}, layout=slider_layout)
chirpRfFreqSlope = widgets.FloatSlider(value=18.39, min=-399, max=399, step=0.01, 
                                       description='chirpRfFreqSlope (MHz/us)', 
                                       style={'description_width': 'initial'}, layout=slider_layout)
chirpRfFreqStart = widgets.FloatSlider(value=76, min=60, max=90, step=0.01, 
                                       description='chirpRfFreqStart (GHz)', 
                                       style={'description_width': 'initial'}, layout=slider_layout)

# Assuming powers_of_two and min_power are already defined
numOfAdcSamples = widgets.SelectionSlider(
    style={'description_width': 'initial'},
    options=powers_of_two,
    value=128,
    description='numOfAdcSamples:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    layout=slider_layout
)


# Interactive widget to update and display values
interact(
    update_parameters,
    chirpRfFreqSlope=chirpRfFreqSlope,
    chirpRfFreqStart=chirpRfFreqStart,
    numOfAdcSamples=numOfAdcSamples,
    chirpRampEndTime=chirpRampEndTime,
    chirpIdleTime=chirpIdleTime,
    chirpAdcSkipSamples=chirpAdcSkipSamples,
    numOfChirppsInBurst=numOfChirppsInBurst,
    numOfBurstsInFrame=numOfBurstsInFrame,
    framePeriodicity=framePeriodicity,
    burstPeriodicity=burstPeriodicity,
    digOutputSampRate=digOutputSampRate
)


interactive(children=(FloatSlider(value=18.39, description='chirpRfFreqSlope (MHz/us)', layout=Layout(height='…

<function __main__.update_parameters(chirpRfFreqSlope, chirpRfFreqStart, numOfAdcSamples, chirpRampEndTime, chirpIdleTime, chirpAdcSkipSamples, numOfChirppsInBurst, numOfBurstsInFrame, framePeriodicity, burstPeriodicity, digOutputSampRate)>